This is a neat project I made as POC for a client. Client was dealing with very dirty data in a sql db, basically every column was a string/varchar. we needed a way to sniff out patterns. Client had tried numerous tools, but all were very slow. billion+ rows, per table, per db. :/

I used a kind of broken makarov chain to generate a pattern of the data, like the reverse of a regex. For example, if a column was a list of ID's, that looked like:
m1234
m9876576a35
m34987
m340598
we see a pattern like M[number][number][number]. M always appears first, but after that its about 4 or so 
random numbers. 

While not the most thorough model, it was more than sufficient when combined with internal domain knowledge to start standing up better documentation. For instance.... if an unknown column came back with a pattern of a[number][number]9, the super users knew this was a document ID for some fax system. I had no clue, but it helped them immensely to figure out what database tables had what data at a quick glance. Given the speed of the algo, it could be ran on lots of stuff, quickly. 

This is NOT the final version. This is like, a working session version, stumbling through the ask.

In [ ]:
import pandas as pd
import numpy as np
import string
from dateutil.parser import parse

In [ ]:
meteordf = pd.read_csv('Meteorite_Landings.csv')

In [ ]:
for x,y in meteordf.dtypes.items():
    print(x,y)

setup our resulting data frame

In [ ]:
dftypes = pd.DataFrame(meteordf.dtypes.items())
dftypes.rename(columns={ dftypes.columns[0]: "column_name", dftypes.columns[1]: "column_type" }, inplace = True)

In [ ]:
dftypes = dftypes.assign(**{'column_agg_type' : 'Pandas Detection' ,  'value_desc' : 'Pandas Inferred Data Type From Load'})

In [ ]:
#variable setup
alphabet = string.ascii_letters+string.punctuation


#make copy of data
recordsdf = meteordf.copy(deep=True)

#attempt to convert everything to a string, sorta helps
all_columns = list(recordsdf)
recordsdf[all_columns] = recordsdf[all_columns].astype(str).replace('nan',np.nan)

#record list of column names
generated_column_names = []

#aggregated_features_df = pd.DataFrame(columns=['column_name', 'column_agg_type', 'value', 'value_desc'])

start with a ton of defintions

In [ ]:

#pandas series checks don't include floats
#need to make a function that does this for us
def is_float(x):
    try:
        float(x)
        return True
    except:
        return False

#pandas has no datetime checker in series methods
#almost any float will come back as true    
def is_date(x):
    try:
        parse(x)
        return True
    except:
        return False

#this is used to try to find a pattern in the column
#assumes most entries are close in look
#will miss patterns if left of string differs

def is_pattern(x):

    pattern_list=[]

    try:
        pattern_df = x.astype(str).apply(lambda x: pd.Series(list(x)))


        pattern_df.dropna(axis='columns', inplace=True)

        n = pd.DataFrame(pattern_df.nunique(axis=0, dropna=True), columns=['count'])

        actualpattern_df = n.loc[n['count'] == 1]


        for x,y in actualpattern_df.iterrows():
            string = "{} is repeated in position {}".format(pattern_df[x].iloc[0],x)
            pattern_list.append(string)

        if not pattern_list:
            pattern_list.append('none')

        return(pattern_list)

    except:
        pattern_list.append('none')
        return(pattern_list)



def statistical_generative_regex(c):

    def add_raw_char(currentcolumnkey, currentchar):
        if attribdct[currentcolumnkey]['raw'].get(currentchar) is None:
            attribdct[currentcolumnkey]['raw'][currentchar]=1
        else:
            attribdct[currentcolumnkey]['raw'][currentchar] = attribdct[currentcolumnkey]['raw'][currentchar] + 1

    length = 0
    totallength = 0
    count = 0
    lengthls = []

    for x in c:
        length = len(str(x))
        totallength = totallength + length
        count = count + 1
        lengthls.append(length)

    #print(np.median(lengthls))

    medianrecordsdf = c[c.astype(str).map(len) == np.ceil(np.median(lengthls))]

    #print(medianrecordsdf)


    medianrecordsdf = medianrecordsdf.astype(str).replace('nan', "")
    medianrecordsdf = medianrecordsdf.astype(str).replace(np.nan, "")

    pattern_df = medianrecordsdf.astype(str).apply(lambda x: pd.Series(list(x)))
    all_columns = list(pattern_df)
    #pattern_df[all_columns] = pattern_df[all_columns].astype(str).replace("", 'null')
    #pattern_df[all_columns] = pattern_df[all_columns].astype(str).replace(np.nan, 'null')

    for cols in all_columns:
        #pattern_df[cols] = pattern_df[cols].astype(str).replace('', 'null')
        pattern_df[cols].mask(pattern_df[cols].astype(str) == '', 'null', inplace=True)
        pattern_df[cols].mask(pattern_df[cols].astype(str) == np.nan, 'null', inplace=True)
        pattern_df[cols].mask(pattern_df[cols].astype(str) == 'nan', 'null', inplace=True)
        pattern_df[cols].mask(pattern_df[cols].astype(str).isnull() == True, 'null', inplace=True)
        pattern_df[cols].fillna('null', inplace=True)


    rowcount = len(pattern_df.index)

    alpha = 0
    digit = 0
    special = 0
    null = 0
    space = 0
    unknown = 0
    unknownls = []
    attribdct = {}

    for x in pattern_df.columns:
        #print(x)

        attribdct[x] = {'agg': {} , 'raw':{}}

        alpha = 0
        digit = 0
        special = 0
        null = 0
        space = 0
        unknown = 0
        for y in pattern_df[x]:
            #print(type(y))
            if y =='null':
                null = null + 1
                add_raw_char(x,y)
            elif y.isalpha():
                alpha = alpha+1
                add_raw_char(x,y)
            elif y.isdigit():
                digit = digit+1
                add_raw_char(x,y)
            elif y in string.punctuation:
                special = special + 1
                add_raw_char(x,y)
            elif y.isspace():
                space = space + 1
                add_raw_char(x,y)
            else:
                unknown = unknown + 1
                unknownls.append(y)
                add_raw_char(x,y)


        #print('Column {}; alpha:{} digit:{} special:{} null:{} space:{} unknown:{} '.format(x, alpha, digit, special, null, space, unknown))


        attribdct[x]['agg'] = {'alpha': alpha, 'digit': digit, 'special': special, 'null': null, 'space': space, 'unknown': unknown}

    outputholder = []
    for z in attribdct:
        charposition = max(attribdct[z]['agg'], key=attribdct[z]['agg'].get)
        charvalue = max(attribdct[z]['raw'], key=attribdct[z]['raw'].get)
        chardesc = charposition
        if attribdct[z]['raw'][charvalue]/rowcount > .90:
            chardesc = charvalue

        formatedchardesc = '[{}]'.format(chardesc)

        outputholder.append(formatedchardesc)

    #print('position {} was a {}'.format(z, position ))

    patternholder = ""

    for chars in outputholder:
        patternholder = patternholder + chars

    return patternholder
    #print(*outputholder)

In [ ]:

#loop each column and run battery of tests
cols_to_iterate = recordsdf.columns
ls = []
for x in cols_to_iterate:
    print(x)

    
    #pull pandas type find
    type_dict = {
        'column_name' : x,
        'column_agg_type':  'Pandas Detection',
        'value' : dftypes.loc[dftypes['column_name'] == x][:1]['column_type'].item().name,
        'value_desc' : 'Pandas Inferred Data Type From Load'
    }
    
    ls.append(type_dict)

    #is alpha test
    newcolumnname = x + '_is_alpha'
    recordsdf[newcolumnname] = recordsdf[x].str.isalpha()
    recordsdf[newcolumnname].fillna(False, inplace = True)

    alpha_dict = {'column_name': x , 'column_agg_type' : 'isalpha' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True"}
    #aggregated_features_df[x]['alpha'] = recordsdf[newcolumnname].mean()
    
    ls.append(alpha_dict)

    #is alphanumeric test
    newcolumnname = x + '_is_alphanumeric'
    recordsdf[newcolumnname] = recordsdf[x].str.isalnum()
    recordsdf[newcolumnname].fillna(False, inplace = True)
    
    numeric_dict = {'column_name': x , 'column_agg_type' : 'isalphanumeric' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True"}

    ls.append(numeric_dict)

    #is_didigt test
    newcolumnname = x + '_is_digit'
    recordsdf[newcolumnname] = recordsdf[x].str.isdigit()
    recordsdf[newcolumnname].fillna(False, inplace = True)
    
    digit_dict = {'column_name': x , 'column_agg_type' : 'isdigit' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True"}
    
    ls.append(digit_dict)

    #is space test
    newcolumnname = x + '_is_space'
    recordsdf[newcolumnname] = recordsdf[x].str.isspace()
    recordsdf[newcolumnname].fillna(False, inplace = True)
    
    space_dict = {'column_name': x , 'column_agg_type' : 'isspace' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True"}
    
    ls.append(space_dict)

    # is lower test
    newcolumnname = x + '_is_lower'
    recordsdf[newcolumnname] = recordsdf[x].str.islower()
    recordsdf[newcolumnname].fillna(False, inplace = True)
    
    lower_dict = {'column_name': x , 'column_agg_type' : 'islower' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True"}
    
    ls.append(lower_dict)

    #is upper test
    newcolumnname = x + '_is_upper'
    recordsdf[newcolumnname] = recordsdf[x].str.isupper()
    recordsdf[newcolumnname].fillna(False, inplace = True)
 
    upper_dict = {'column_name': x , 'column_agg_type' : 'isupper' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True"}

    #numeric test
    newcolumnname = x + '_is_numeric'
    recordsdf[newcolumnname] = recordsdf[x].str.isnumeric()
    recordsdf[newcolumnname].fillna(False, inplace = True)
    
    numerical_dict = {'column_name': x , 'column_agg_type' : 'isnumeric' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True"}
    ls.append(numerical_dict)

    #decimal test
    newcolumnname = x + '_is_decimal'
    recordsdf[newcolumnname] = recordsdf[x].str.isdecimal()
    recordsdf[newcolumnname].fillna(False, inplace = True)
    decimal_dict = {'column_name': x , 'column_agg_type' : 'isdecimal' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True"}
    ls.append(numerical_dict)

    #trying float logic
    newcolumnname = x + '_is_float'
    recordsdf[newcolumnname] = recordsdf[x].apply(is_float)
    recordsdf[newcolumnname].fillna(False, inplace = True)
    float_dict = {'column_name': x , 'column_agg_type' : 'isfloat' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True"}
    ls.append(float_dict)


    #trying date logic
    newcolumnname = x + '_is_date'
    recordsdf[newcolumnname] = recordsdf[x].apply(is_date)
    recordsdf[newcolumnname].fillna(False, inplace = True)
    
    date_dict = {'column_name': x , 'column_agg_type' : 'isdate' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True(High False Postive)"}
    ls.append(date_dict)


    #special character test
    newcolumnname = x + '_is_specialcharacters'
    recordsdf[newcolumnname] = recordsdf[x].str.strip(alphabet).astype(bool)
    special_char_test = {'column_name': x , 'column_agg_type' : 'isspecialcharacters' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True"}
    ls.append(special_char_test)

    #is_na test
    newcolumnname = x + '_is_na'
    recordsdf[newcolumnname] = recordsdf[x].isnull()
    recordsdf[newcolumnname].fillna(False, inplace = True)
    
    na_dict = {'column_name': x , 'column_agg_type' : 'isna' , 'value': recordsdf[newcolumnname].mean(), 'value_desc' : "% True"}
    ls.append(na_dict)

    #min test
    newcolumnname = x + '_min'
    recordsdf[newcolumnname] = recordsdf[x].astype('string').min()
    recordsdf[newcolumnname].fillna(False, inplace = True)
    
    min_dict = {'column_name': x , 'column_agg_type' : 'min' , 'value': recordsdf[newcolumnname].iloc[0], 'value_desc' : "min"}
    ls.append(min_dict)

    #max test
    newcolumnname = x + '_max'
    recordsdf[newcolumnname] = recordsdf[x].astype('string').max()
    recordsdf[newcolumnname].fillna(False, inplace = True)
    max_dict = {'column_name': x , 'column_agg_type' : 'max' , 'value': recordsdf[newcolumnname].iloc[0], 'value_desc' : "max"}
    ls.append(max_dict)

    #statistical
    newcolumnname = x + '_statistical regex'
    pattern_return = statistical_generative_regex(recordsdf[x])
    recordsdf[newcolumnname] = str(pattern_return)
    recordsdf[newcolumnname].fillna(False, inplace = True)
    
    stat_dict = {'column_name': x , 'column_agg_type' : 'statistical_regex' , 'value': recordsdf[newcolumnname].iloc[0], 'value_desc' : "list"}
    ls.append(stat_dict)
    
    
        
    
aggregated_features_df = pd.DataFrame(ls)
    
    